<a href="https://colab.research.google.com/github/kevingray92/597_Project4_GroupB/blob/dailyobscleanup/Project4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import os
import tarfile

In [0]:
#Download the data
!wget https://raw.githubusercontent.com/swnesbitt/ATMS-597-SP-2020/master/ATMS-597-SP-2020-Project-4/KCMI_hourly.csv

In [0]:
#Used to unzip the .tar.gz files
!gunzip drive/My\ Drive/Colab\ Notebooks/ATMS-597/Module04/Project04/sfc.tar.gz

In [0]:
#Use this to untar the .tar files
#Looks like it will send all of these to a new remote directory called 'bufkit'
#Might want to specify a destination folder in the future
!tar -xvf drive/My\ Drive/Colab\ Notebooks/ATMS-597/Module04/Project04/sfc.tar

In [3]:
dailyobs = pd.read_csv('drive/My Drive/KCMI_dailyNOFOOTER.csv', header=8) #Read the daily obs (removed the footer in the original file)
dailyobs.drop(['Total Precip (in)'],axis=1) #Drop the daily precip column that we are going to replace
dailyobs

,Date,Max Hourly Temp (F),Min Hourly Temp (F),Max Wind Speed (mph),Total Precip (in)
0,1/1/2010,12,2,15,M
1,1/2/2010,6,-5,15,0
2,1/3/2010,10,-6,11,0
3,1/4/2010,11,-2,20,0
4,1/5/2010,10,-1,16,M
...,...,...,...,...,...
3647,12/27/2019,43,32,15,0
3648,12/28/2019,56,32,23,0.23
3649,12/29/2019,59,41,23,0.22
3650,12/30/2019,38,31,26,0.01


In [4]:
hourlyobs = pd.read_csv('drive/My Drive/KCMI_hourly.csv') #Read the hourly file
hourlyprecip = hourlyobs.drop(['year','month','day','hour','tmpc','dwpc','mslp','wdir','wspd','skct','pr6h','doy','woy'],axis=1) #Only keep precip column
hourlyprecip.set_index('Timestamp') #Make the index be the timestamp
hourlyprecip.index = pd.to_datetime(hourlyprecip['Timestamp']) #Make the index be a datetime
dailyprecip = hourlyprecip.resample('d').sum() #Resample to daily
dailyprecip.columns = ['Daily Precip (mm)'] #Rename the column
dailyprecip

,Daily Precip (mm)
Timestamp,
2005-12-31,0.0
2006-01-01,0.0
2006-01-02,2.3
2006-01-03,0.2
2006-01-04,0.0
...,...
2020-03-05,0.0
2020-03-06,-0.2
2020-03-07,0.0


In [6]:
dailyprecipshort = dailyprecip[(dailyprecip.index>'2009-12-31') & (dailyprecip.index<'2020-01-01')] #Only get 2010 to 2019
dailyobs['Daily Precip (mm)'] = dailyprecipshort['Daily Precip (mm)'].values #Put the new calculated daily precip in the dailyobs dataframe
dailyobs #This should be the final dataframe used as the predictands (!!!!!Might want to clean up negative precip values later!!!!!)

,Date,Max Hourly Temp (F),Min Hourly Temp (F),Max Wind Speed (mph),Total Precip (in),Daily Precip (mm)
0,1/1/2010,12,2,15,M,0.0
1,1/2/2010,6,-5,15,0,0.0
2,1/3/2010,10,-6,11,0,0.0
3,1/4/2010,11,-2,20,0,0.0
4,1/5/2010,10,-1,16,M,-0.1
...,...,...,...,...,...,...
3647,12/27/2019,43,32,15,0,0.0
3648,12/28/2019,56,32,23,0.23,2.7
3649,12/29/2019,59,41,23,0.22,4.0
3650,12/30/2019,38,31,26,0.01,0.4
